In [1]:
# импортируем библиотеки pandas и numpy
import pandas as pd
import numpy as np

In [2]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/StateFarm.csv', sep=';')
data.head(3)

,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Response
0,18975.456110,65999,237,1,14,0,6,0
1,4715.321344,0,65,19,56,0,3,0
2,5018.885233,54500,63,28,17,0,6,0


In [3]:
# импортируем функцию train_test_split(), с помощью
# которой разбиваем данные на обучающие и тестовые
from sklearn.model_selection import train_test_split
# разбиваем данные на обучающие и тестовые: получаем обучающий
# массив признаков, тестовый массив признаков, обучающий массив
# меток, тестовый массив меток
X_train, X_test, y_train, y_test = train_test_split(data.drop('Response', axis=1), 
                                                    data['Response'], 
                                                    test_size=0.3,
                                                    stratify=data['Response'],
                                                    random_state=42)

In [4]:
# импортируем необходимые библиотеки
import h2o
import os
h2o.init(nthreads=-1, max_mem_size=8)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_202"; Java(TM) SE Runtime Environment (build 1.8.0_202-b08); Java HotSpot(TM) 64-Bit Server VM (build 25.202-b08, mixed mode)
  Starting server from /anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/y_/s7c_myjd7qg6zs3hcfflpgwr0000gn/T/tmpr5tnph0d
  JVM stdout: /var/folders/y_/s7c_myjd7qg6zs3hcfflpgwr0000gn/T/tmpr5tnph0d/h2o_artemgruzdev_started_from_python.out
  JVM stderr: /var/folders/y_/s7c_myjd7qg6zs3hcfflpgwr0000gn/T/tmpr5tnph0d/h2o_artemgruzdev_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Europe/Moscow
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.5
H2O cluster version age:,2 months and 22 days
H2O cluster name:,H2O_from_python_artemgruzdev_wli1l6
H2O cluster total nodes:,1
H2O cluster free memory:,7.111 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"accepting new members, healthy"


In [5]:
# конкатенируем обучающие массив признаков 
# и массив меток
train = pd.concat([X_train, y_train], axis=1)
# конкатенируем тестовые массив признаков 
# и массив меток
test = pd.concat([X_test, y_test], axis=1)
# преобразовываем датафреймы pandas во фреймы h2o
tr = h2o.H2OFrame(train)
tst = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
# смотрим содержимое фрейма
tr.describe()

Rows:5783
Cols:8




,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Response
type,real,int,int,int,int,int,int,int
mins,1898.007675,0.0,61.0,0.0,0.0,0.0,1.0,0.0
mean,8032.769846154935,37840.9216669549,93.14611793186961,15.12588621822582,48.13695313850944,0.38422963859588494,2.9332526370395993,0.10012104444060177
maxs,83325.38119,99981.0,298.0,35.0,99.0,5.0,9.0,1.0
sigma,7031.409016463689,30652.71816091171,34.3799942891798,10.104644065491298,27.719663032389967,0.9109656865217554,2.3682231143012142,0.30018728020055196
zeros,0,1486,0,200,46,4604,0,5204
missing,0,0,0,0,0,0,0,0
0,12200.78537,60282.0,102.0,23.0,87.0,0.0,2.0,0.0
1,5330.593183,84240.0,132.0,10.0,51.0,0.0,1.0,0.0
2,10426.08551,52342.0,66.0,28.0,28.0,3.0,2.0,0.0


In [7]:
# преобразовываем в категориальную переменную
tr['Response'] = tr['Response'].asfactor()
tst['Response'] = tst['Response'].asfactor()

In [8]:
# задаем имя зависимой переменной
dependent = 'Response'
# задаем список имен предикторов
predictors = list(tr.columns)
# удаляем имя зависимой переменной 
# из списка имен предикторов
predictors.remove(dependent)

In [9]:
# импортируем класс H2OGeneralizedLinearEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
# создаем экземпляр класса H2OGeneralizedLinearEstimator
glm_model = H2OGeneralizedLinearEstimator(family='binomial')
# обучаем модель
glm_model.train(x = predictors, y = dependent, 
                training_frame=tr, validation_frame=tst)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [11]:
# вычисляем спрогнозированные значения и спрогнозированные 
# вероятности классы зависимой переменной
predictions = glm_model.predict(tst)
predictions

glm prediction progress: |████████████████████████████████████████████████| 100%


predict,p0,p1
0,0.910308,0.0896917
0,0.89851,0.10149
1,0.88301,0.11699
0,0.902297,0.0977026
0,0.925552,0.0744484
0,0.904464,0.0955355
1,0.880408,0.119592
1,0.883242,0.116758
0,0.905155,0.0948445
1,0.878944,0.121056
